In [2]:
from os import getenv
from dotenv import load_dotenv
from pinecone import Pinecone

load_dotenv()
pc = Pinecone(api_key=getenv('PINECONE_API_KEY'))
index = pc.Index('nano-sous-chef')

In [3]:
from langchain_ollama import ChatOllama

creative_llm = ChatOllama(
    model="llama3.2",
    temperature=0.5
)

strict_llm = ChatOllama(
    model="llama3.2",
    temperature=0
)

In [ ]:
creative_llm.invoke('What is a train?')

In [5]:
import sqlite3

db = sqlite3.connect('cookbook.db')
cursor = db.cursor()

In [6]:
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import ChatPromptTemplate

# ingredients chain
ingredients_system_prompt = '''
You are a chef's digital assistant, and you were just provided with some text.
Extract a list of all of the ingredients mentioned and return it structured in JSON format.
I only want the name of the ingredient.
Make sure to capitalize the first letter of every word.
'''

ingredients_prompt_template = ChatPromptTemplate.from_messages([
    ("system", ingredients_system_prompt),
    ("user", "{user_prompt}")
])

ingredients_chain = ingredients_prompt_template | strict_llm | JsonOutputParser()

def ingredients_node(src):
    return ingredients_chain.invoke({'user_prompt' : src})['ingredients']

# example
# print(ingredients_node('I have eggs, cabbages, and strawberries in my pantry.'))

In [7]:
from langchain_core.output_parsers import StrOutputParser

# exit station
terminal_system_prompt = '''
Determine whether the user would like to 'try again' or 'exit', based on the input given.
Categorize what they said as either one, and only return that answer. If the user doesn't seem to reflect
intent to try again, respond with 'exit' regardless.
'''

terminal_prompt_template = ChatPromptTemplate.from_messages([
    ("system", terminal_system_prompt),
    ("user", "{user_prompt}")
])

terminal_chain = terminal_prompt_template | strict_llm | StrOutputParser()

def terminal_node(negative_outcome=False):
    if negative_outcome:
        print('\n\nSorry I wasn\'t able to help with that last query! Would you like to try again or exit?\n')
    else:
        print('\n\nThanks for trying out my system! Would you like to try again or exit?\n')
    response = input()
    intent = terminal_chain.invoke({'user_prompt' : response})
    if intent == 'try again':
        print('Wants to try again, not implemented yet!')
        # TODO invoke genesis from here

In [8]:
# full description chain
def collect_information(id):
    res = []
    cursor.execute('''
        SELECT name, aggregated_rating, recipe_servings, recipe_yield, recipe_instructions,
            cook_time, prep_time, total_time FROM information where id = {};
                   '''.format(id))
    res.append(cursor.fetchone())
    cursor.execute('''
        SELECT * from macros WHERE recipe_id={};
                   '''.format(id))
    res.append(cursor.fetchone())
    cursor.execute('''
        SELECT * from ingredient_recipe WHERE recipe_id={};
                   '''.format(id))
    res.append(cursor.fetchall())
    return res

full_description_system_prompt = '''
You are a chef's digital assistant, and you were just provided with text that has all the information
about a particular dish. Using the information provided, provide a detailed description of the dish itself,
before describing how to make it as per the information provided.

Structure your response as if it's an entry in a recipe book, and make it clear and easy to follow.
'''

full_description_prompt_template = ChatPromptTemplate.from_messages([
    ("system", full_description_system_prompt),
    ("user", "{dish_info}")
])

full_description_chain = full_description_prompt_template | creative_llm

def full_description_node(id):
    info = collect_information(id)
    # res = full_description_chain.invoke({'dish_info' : info})
    # print(res, '\n')

    for chunk in full_description_chain.stream({'dish_info' : info}):
        print(chunk.content, end='', flush=True)

    terminal_node()

In [9]:
def trim_description_header(desc):
    return desc.split('] ')[1]

def retrieve_description(index):
    cursor.execute(f"SELECT generated_description FROM generated_descriptions WHERE recipe_id = {index}")
    return trim_description_header(cursor.fetchone()[0])

def retrieve_name(index):
    cursor.execute(f"SELECT name FROM information WHERE id = {index}")
    return cursor.fetchone()[0]

def generate_dish_list(dishes):
    res = ''
    i = 1
    for id, (name, desc) in dishes.items():
        res += (f"{i}. {name} - {desc}\n")
        i += 1
    return res

summarization_categorization_system_prompt = '''
You are a chef's digital assistant. The user will provide a list of dishes, followed by a message.
Determine the intent behind the message provided, and categorize it as follows:

If the user wants to end the run or exit, answer 'exit'.
If the user wants to see a full description of a specific dish, return 'full description'.

Only answer in the format instructed, and don't have quotes as part of the answer
'''

dish_picker_system_prompt = '''
You are a chef's digital assistant. You'll recieve what a user said, and a list of dishes they picked out from.
Return the exact name of the dish they picked out from the options provided. If their response is not clear, return 'unclear'.

Only provide either option and nothing else.
'''

summarization_categorization_prompt_template = ChatPromptTemplate.from_messages([
    ("system", summarization_categorization_system_prompt),
    ("user", "{user_prompt}")
])

dish_picker_prompt_template = ChatPromptTemplate.from_messages([
    ("system", dish_picker_system_prompt),
    ("user", "#DISH LIST#\n{dish_list}\n\nUSER MESSAGE: {user_prompt}")
])

summarization_categorization_chain = summarization_categorization_prompt_template | strict_llm | StrOutputParser()
dish_picker_chain = dish_picker_prompt_template | strict_llm | StrOutputParser()

def summarization_node(src):
    # a list of ids and their score received
    # sort by relevance and remove duplicates
    id_to_max_relevance = dict()
    for id, relevance in src:
        if id not in id_to_max_relevance:
            id_to_max_relevance[id] = relevance
        else:
            id_to_max_relevance[id] = max(id_to_max_relevance[id], relevance)
    unique_src = [(k, v) for k, v in id_to_max_relevance.items()]
    unique_src.sort(key=lambda x: x[1])
    match_ids = [x[0] for x in unique_src]

    # pick the top three and retrieve their entries from the RDB.
    picks = match_ids[-3:-1]
    id_to_name_and_description = dict()
    for id in picks:
        id_to_name_and_description[id] = (retrieve_name(id), retrieve_description(id))

    # output section
    print("Here are some dishes that come to mind:\n\n")
    dish_list = generate_dish_list(id_to_name_and_description)
    print(dish_list)
    print("Would you like to see a full description of any of these, or end this run?\n")

    user_response = input()
    intent = summarization_categorization_chain.invoke({'user_prompt':user_response, 'dish_list':dish_list})
    if intent == 'exit':
        terminal_node()
    else:
        picked_dish_name = dish_picker_chain.invoke({'user_prompt':user_response, 'dish_list':dish_list})
        if picked_dish_name == 'unclear':
            print('I couldn\'t find that dish.\n')
            terminal_node(negative_outcome=True)
            return

        picked_dish_id = -1
        for id, (name, desc) in id_to_name_and_description.items():
            if name == picked_dish_name:
                picked_dish_id = id
                break
        if picked_dish_id == -1:
            print('I couldn\'t find that dish.\n')
            terminal_node(negative_outcome=True)
            return
        full_description_node(picked_dish_id)


In [10]:
def fix_vector(v):
    return [float(x) for x in v]

def find_matches(src_embeddings):
    src_embeddings = [fix_vector(v) for v in src_embeddings]
    query_responses = []
    for v in src_embeddings:
        response = index.query(
                namespace="descriptions",
                vector=v,
                top_k=3,
                include_values=False,
            )
        list_form = [(match['id'], match['score']) for match in response['matches']]
        query_responses.extend(list_form)
    return query_responses

def match_retrieval_node(src):
    matches = find_matches(src)

    if len(matches) == 0:
        print('I don\'t know a dish that matches that description, let\'s start over')
        terminal_node(negative_outcome=True)
        return

    # generate the response for the dishes found
    summarization_node(matches)

In [ ]:
from sentence_transformers import SentenceTransformer
embeddings_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
def generate_embeddings_for_generated_descriptions(generated_descriptions):
    return list(embeddings_model.encode(generated_descriptions))

def embeddings_generation_node(src):
    res = generate_embeddings_for_generated_descriptions(src)

    match_retrieval_node(res)

In [ ]:
# description generation phase chain

description_generation_system_prompt = '''
You are a chef's digital assistant. You'll be provided with a description of a dish.
I want you to generate exactly 3 similar descriptions to the one provided.
Make sure that there are exactly 3 generated descriptions.
Return the descriptions in JSON. It should be in the format below:
descriptions : [x,y,z]
'''
description_generation_prompt_template = ChatPromptTemplate.from_messages([
    ("system", description_generation_system_prompt),
    ("user", "{user_prompt}")
])

description_generation_chain = description_generation_prompt_template | creative_llm | JsonOutputParser()

def description_generation_helper(src):
    res = description_generation_chain.invoke({'user_prompt' : src})
    return res['descriptions'] + [src]

def description_generation_node(src):
    res = description_generation_helper(src)

    # send to embeddings generation
    embeddings_generation_node(res)

# example
description_generation_node(
    'The dish I have in mind is biryani.'
)

In [13]:
from langchain_core.output_parsers import StrOutputParser

# genesis chain
genesis_system_prompt = '''
You are a chef's digital assistant. You'll recieve a prompt from the chef and your
job is to classify the prompt into the category it fits into.

The category names are:
dish description
ingredient list

Respond with only the category name.
'''

genesis_prompt_template = ChatPromptTemplate.from_messages([
    ("system", genesis_system_prompt),
    ("user", "{user_prompt}")
])

genesis_chain = genesis_prompt_template | strict_llm | StrOutputParser()

# example case
# print(genesis_chain.invoke({'user_prompt':'tuna'}))
# print(genesis_chain.invoke({'user_prompt':'tuna sandwich'}))

genesis_opener = '''
Hey, I\'m nano sous chef. Let\'s create an amazing dish!

You can either give me the name or description of the dish you have in mind,
or a list of what ingredients you have so I can suggest some possible dishes from what I know!\n
'''

def genesis_node():
    print(genesis_opener)
    user_input = input()
    if genesis_chain.invoke(user_input) == 'ingredient list':
        ingredients_node(user_input)
    else:
        description_generation_node(user_input)
# example
# print(genesis_node())

In [ ]:
genesis_node()